<a href="https://colab.research.google.com/github/Teasotea/DialogSystem/blob/main/ConversationalAI_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import Libraries

In [1]:
!pip install transformers

     |████████████████████████████████| 4.2 MB 6.8 MB/s 
     |████████████████████████████████| 6.6 MB 49.4 MB/s 
     |████████████████████████████████| 84 kB 2.8 MB/s 
     |████████████████████████████████| 596 kB 48.0 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
!pip install nltk

In [3]:
import numpy as np
import pandas as pd
import time
import os
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
import nltk
from sklearn.svm import OneClassSVM
import re

In [4]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.metrics import f1_score

# Part I: Classification

## Basic Preprocessing

In [5]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import re
nltk.download('stopwords')
nltk.download('punkt')
stop_words = set(stopwords.words("english"))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [6]:
clean_sent = lambda sent: ' '.join([s.lower() for s in pd.Series(sent).str.replace('[^\w\s]', '')])

In [7]:
preprocess = lambda text:clean_sent(' '.join([w for w in word_tokenize(text) if not w.lower() in stop_words])).strip()

In [8]:
preprocess('what does linear regression sTands for ???')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  """Entry point for launching an IPython kernel.


'linear regression stands'

## Import models for One-Class Classification with SVM

In [9]:
from google.colab import drive
from pathlib import Path
from google.colab import drive

drive.mount('/content/gdrive')

GDRIVE = '/content/gdrive/MyDrive/chatbot'
WORKDIR = GDRIVE + '/classif/'
Path(WORKDIR).mkdir(parents=True, exist_ok=True)

%cd  '/content/gdrive/MyDrive/chatbot/classif'

Mounted at /content/gdrive
/content/gdrive/MyDrive/chatbot/classif


In [10]:
model_bye = pd.read_pickle(WORKDIR+"model_bye.pkl")

In [11]:
model_greet = pd.read_pickle(WORKDIR+ "model_greet.pkl")

In [12]:
tokenizer_greet = pd.read_pickle(WORKDIR+ "tokenizer_greet.pkl")

In [15]:
tokenizer_bye = pd.read_pickle(WORKDIR+ "tokenizer_bye.pkl")

In [13]:
num_words = 1000
oov_token = '<UNK>'
pad_type = 'post'
trunc_type = 'post'
maxlen = 10

In [16]:
def classify_goodbyes(sent):
  l = [preprocess(sent)]
  test_sequences = tokenizer_bye.texts_to_sequences(l)
  test_padded= pad_sequences(test_sequences, padding=pad_type, truncating=trunc_type, maxlen=maxlen)
  pred_class = model_bye.predict(test_padded)
  cl = int(*pred_class)
  return  cl
  
print(classify_goodbyes('Bye, bot'))

1


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  """Entry point for launching an IPython kernel.


In [17]:
def classify_greet(sent):
  l = [preprocess(sent)]
  test_sequences = tokenizer_greet.texts_to_sequences(l)
  test_padded= pad_sequences(test_sequences, padding=pad_type, truncating=trunc_type, maxlen=maxlen)
  pred_class_greet = model_greet.predict(test_padded)
  cl = int(*pred_class_greet)
  return  cl

In [18]:
print(classify_greet('Hi, bot'))

1


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  """Entry point for launching an IPython kernel.


# Part II: Question Answering

## Data Science Interview Dataset

In [19]:
qa_data = pd.read_csv('https://raw.githubusercontent.com/Kizuna-Cheng/Data_Science_Interviews_NLP/main/data.csv')
qa_data

,Category,Questions,Answers
0,Linear Regression,What are the assumptions required for linear r...,There are four assumptions associated with a l...
1,Statistics,What is collinearity? What is multicollinearit...,Collinearity is a linear association between t...
2,Linear Regression,What are the drawbacks of a linear model?\n,There are a couple of drawbacks of a linear mo...
3,Linear Regression,What are ridge and lasso regression and what a...,Both L1 and L2 regularization are methods used...
4,KNN,How does K-Nearest Neighbor work?,K-Nearest Neighbors is a classification techni...
...,...,...,...
319,Neural Network,What are advantages and disadvantages of using...,Advantages:\n\nStore data on the entire networ...
320,Decision Trees,What is the default method for splitting in de...,"The default method is the Gini Index, which is..."
321,"XGBoost, SVM",Why does XGBoost perform better than SVM?,XGBoos is an ensemble method that uses many tr...
322,Neural Network,"For NLP, what’s the main purpose of using an e...",We use the encoder-decoder model to generate a...


In [158]:
qa_data['Category'].unique()

array(['Linear Regression', 'Statistics ', 'KNN', 'KMeans', 'Naive Bayes',
       'SVM', 'Decision Trees', 'Random Forest, Naive Bayes',
       'Random Forest, SVM', 'AdaBoost, Gradient Boost', 'EDA',
       'Methodology', 'XGBoost', 'Neural Network', 'Statistics',
       'Time Series, Decision Trees', 'KNN, KMeans',
       'Random Forest, Gradient Boosting', 'Random Forest', 'Time Series',
       'KMeans, KNN', 'Linear Regression, Logistic Regression',
       'Logistic Regression', 'Gradient Boosting',
       'XGBoost, Gradient Boosting',
       'Gradient Boosting, Logistic Regression', 'KMeans, Methodology',
       'Decision Trees, Time Series', 'XGBoost, SVM'], dtype=object)

In [160]:
qa_data[qa_data['Category'] == 'EDA']

,Category,Questions,Answers
14,EDA,Explain what the bootstrap sampling method is ...,"Technically speaking, the bootstrap sampling m..."
19,EDA,Give several ways to deal with missing values,There are a number of ways to handle null valu...
20,EDA,Is mean imputation of missing data acceptable ...,Mean imputation is the practice of replacing n...
24,EDA,How can you identify outliers?,There are a couple of ways to identify outlier...
25,EDA,What is an inlier?,An inlier is a data observation that lies with...
26,EDA,How can outliers be treated?,There are a couple of ways:\nRemove outliers i...
29,EDA,What is the difference between a validation se...,"Generally, the validation set is used to tune ..."
31,EDA,What are some of the steps for data wrangling ...,There are many steps that can be taken when da...
36,EDA,What are the feature selection methods used to...,There are two types of methods for feature sel...
79,EDA,What is an outlier? Explain how you might scre...,An outlier is a data point that differs signif...


In [148]:
qa_data.Questions[72]

'How to check if the regression model fits the data well?'

In [20]:
qa_data.Questions[:10].tolist()

['What are the assumptions required for linear regression? What if some of these assumptions are violated?',
 'What is collinearity? What is multicollinearity? How do you deal with it?\n',
 'What are the drawbacks of a linear model?\n',
 'What are ridge and lasso regression and what are the differences between them?',
 'How does K-Nearest Neighbor work?',
 'How can you select k for k means?',
 'Why is Naive Bayes “naive”?',
 'What are the support vectors in SVM?',
 'What is pruning in decision trees?',
 'What are random forests? Why is Naive Bayes better?']

In [21]:
qa_data.Answers[:10].tolist()

['There are four assumptions associated with a linear regression model:\nLinearity: The relationship between X and the mean of Y is linear.\nHomoscedasticity: The variance of the residual is the same for any value of X.\nIndependence: Observations are independent of each other.\nNormality: For any fixed value of X, Y is normally distributed.\nExtreme violations of these assumptions will make the results redundant. Small violations of these assumptions will result in a greater bias or variance of the estimate.',
 'Collinearity is a linear association between two predictors. Multicollinearity is a situation where two or more predictors are highly linearly related.\nThis can be problematic because it undermines the statistical significance of an independent variable. While it may not necessarily have a large impact on the model’s accuracy, it affects the variance of the prediction and reduces the quality of the interpretation of the independent variables.\nYou could use the Variance Infla

In [22]:
test_data = [
             'What does linear regression stands for?',
 'What is the differencebetween collinearity and multicollinearity?',
 'What are the cons of using a linear model?\n',
 'What are ridge and lasso regression?',
 'How does K-Nearest neighbor work?',
 'How to select k for k means?',
 'Why is Naive Bayes “naive”?',
 'When should I use SVM?',
'What is pruning in decision trees?',
 'What are random forests? Why is Naive Bayes better?',
 'What\'s collinearity?']

In [23]:
def getResults(questions, fn1, fn2):
    def getResult(q):
        answer, score, prediction = fn1(q, fn2)
        return [q, prediction, answer, score]
    return pd.DataFrame(list(map(getResult, questions)), columns=["Q", "Prediction", "A", "Score"])

## Summarize question
A pegasus [model](https://huggingface.co/google/pegasus-xsum) with sampled gap sentence ratios on both C4 and HugeNews, and stochastically sample important sentences: `google/pegasus-xsum`

In [24]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, AutoConfig
summ_tokenizer = AutoTokenizer.from_pretrained("google/pegasus-xsum")
summ_model = AutoModelForSeq2SeqLM.from_pretrained("google/pegasus-xsum")

Downloading:   0%|          | 0.00/87.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.82M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.36M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.12G [00:00<?, ?B/s]

In [25]:
test_question = "Tell me pls, what does linear regression stands for?"

In [64]:
def get_question_summary(text):
  summ_input_ids = summ_tokenizer.encode(text, return_tensors="pt")
  summ_outputs = summ_model.generate(summ_input_ids)
  summ_decoded = summ_tokenizer.decode(summ_outputs[0], skip_special_tokens=True)
  return(summ_decoded)

In [65]:
get_question_summary(test_question)

'What is linear regression?'

This is wierd. Better to try a different model

In [106]:
get_question_summary("Tell me pls, what are support vector machines?")

'What is the difference between a vector machine and a support vector machine?'

## Text Similarity

In [28]:
def preprocess_text(text):
    text = str(text)
    text = text.lower()

    # Clean the text
    text = re.sub(r"[^A-Za-z0-9^,!.\/'+-=]", " ", text)
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "cannot ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", " ! ", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", " ^ ", text)
    text = re.sub(r"\+", " + ", text)
    text = re.sub(r"\-", " - ", text)
    text = re.sub(r"\=", " = ", text)
    text = re.sub(r"'", " ", text)
    return(text)

### Cosine similarity with [Sentence Transformers ](https://www.sbert.net/docs/pretrained_models.html)
All-round[ model](https://huggingface.co/sentence-transformers/all-mpnet-base-v2) tuned for many use-cases.

Trained on a large and diverse dataset of over 1 billion training pairs `all-mpnet-base-v2`

In [35]:
!pip install -U sentence-transformers

     |████████████████████████████████| 79 kB 4.5 MB/s 
     |████████████████████████████████| 1.2 MB 16.1 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.0-py3-none-any.whl size=120747 sha256=2ae40d96b65086fd17ece2dd2c74ddf3ba89f9068c2b3f93937dd93da3ee5b9f
  Stored in directory: /root/.cache/pip/wheels/83/c0/df/b6873ab7aac3f2465aa9144b6b4c41c4391cfecc027c8b07e7
Successfully built sentence-transformers


In [67]:
from sentence_transformers import SentenceTransformer, util, models
from scipy.spatial import distance

In [37]:
st_model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')

Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/10.1k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/571 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/239 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/363 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/13.1k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

In [38]:
sent_vect = lambda sentences: st_model.encode(sentences)

In [47]:
# all_data_to_vector = sent_vect(qa_data.Questions.tolist())
# pd.to_pickle(all_data_to_vector, WORKDIR+ "sentence-transformers_embeddings_qa_data.pkl")

In [46]:
all_data_vectors = pd.read_pickle(WORKDIR+ "sentence-transformers_embeddings_qa_data.pkl")

In [69]:
all_data_vectors.shape

(324, 768)

In [63]:
# sent_vect([preprocess_text('What is linear regression?')])

In [75]:
def cosine_distance_sbert(s1, v1):
    cosine = distance.cosine(s1.tolist(), v1.tolist())
    cosine_simil = (1-cosine)
    return cosine_simil

In [78]:
 # preprocess_text
 cosine_distance_sbert(sent_vect([preprocess_text('What\'s collinearity?')]), all_data_vectors[1])

0.7552678862823228

In [77]:
# get_question_summary
cosine_distance_sbert(sent_vect([get_question_summary('What\'s collinearity?')]), all_data_vectors[1])

0.8044805008373311

## Get approximate answer

In [81]:
def getApproximateAnswer_cos_sim(q, is_quick = True):
    if is_quick:
      q = sent_vect([preprocess_text(q)])
    else:  q = sent_vect([get_question_summary(q)])
    max_score = 0
    answer = ""
    prediction = ""
    scores = []
    for i,j in enumerate(all_data_vectors):
        score = cosine_distance_sbert(q, j)
        scores.append(score)
        if score >= 0.95:
            return  qa_data.Answers.tolist()[i], score, qa_data.Questions.tolist()[i]
        elif score > max_score:
            max_score = score
            answer = qa_data.Answers.tolist()[i]
            prediction = qa_data.Questions.tolist()[i]
    return answer, max_score, prediction

In [111]:
getApproximateAnswer_cos_sim(preprocess_text('Tell me pls, what does linear regression stands for?'))

('Regression is a part of supervised ML. Regression models investigate the relationship between a dependent (target) and independent variable (s) (predictor). Here are some common regression models\n\nLinear Regression establishes a linear relationship between target and predictor (s). It predicts a numeric value and has a shape of a straight line.\nPolynomial Regression has a regression equation with the power of independent variable more than 1. It is a curve that fits into the data points.\nRidge Regression helps when predictors are highly correlated (multicollinearity problem). It penalizes the squares of regression coefficients but doesn’t allow the coefficients to reach zeros (uses L2 regularization).\nLasso Regression penalizes the absolute values of regression coefficients and allows some of the coefficients to reach absolute zero (thereby allowing feature selection).',
 0.6253195033074085,
 'What is regression? Which models can you use to solve a regression problem?')

In [112]:
getApproximateAnswer_cos_sim(preprocess_text('Tell me pls, what does linear regression stands for?'), False)

('Regression is a part of supervised ML. Regression models investigate the relationship between a dependent (target) and independent variable (s) (predictor). Here are some common regression models\n\nLinear Regression establishes a linear relationship between target and predictor (s). It predicts a numeric value and has a shape of a straight line.\nPolynomial Regression has a regression equation with the power of independent variable more than 1. It is a curve that fits into the data points.\nRidge Regression helps when predictors are highly correlated (multicollinearity problem). It penalizes the squares of regression coefficients but doesn’t allow the coefficients to reach zeros (uses L2 regularization).\nLasso Regression penalizes the absolute values of regression coefficients and allows some of the coefficients to reach absolute zero (thereby allowing feature selection).',
 0.7435179757644976,
 'What is regression? Which models can you use to solve a regression problem?')

In [83]:
getResults(test_data, getApproximateAnswer_cos_sim, cosine_distance_sbert)

,Q,Prediction,A,Score
0,What does linear regression stands for?,What is regression? Which models can you use t...,Regression is a part of supervised ML. Regress...,0.592466
1,What is the differencebetween collinearity and...,What is collinearity? What is multicollinearit...,Collinearity is a linear association between t...,0.828295
2,What are the cons of using a linear model?\n,What are the drawbacks of a linear model?\n,There are a couple of drawbacks of a linear mo...,0.929416
3,What are ridge and lasso regression?,What are ridge and lasso regression and what a...,Both L1 and L2 regularization are methods used...,0.910777
4,How does K-Nearest neighbor work?,How does K-Nearest Neighbor work?,K-Nearest Neighbors is a classification techni...,0.995851
5,How to select k for k means?,How can you select k for k means?,"You can use the elbow method, which is a popul...",0.955217
6,Why is Naive Bayes “naive”?,Why is Naive Bayes “naive”?,Naive Bayes is naive because it holds a strong...,0.967245
7,When should I use SVM?,When would you use random forests Vs SVM and why?,There are a couple of reasons why a random for...,0.683490
8,What is pruning in decision trees?,What is pruning in decision trees?,Pruning is a technique in machine learning and...,0.986298
9,What are random forests? Why is Naive Bayes be...,What are random forests? Why is Naive Bayes be...,Random forests are an ensemble learning techni...,0.975298


## BERT info retrieval

In [84]:
from transformers import BertTokenizer, BertForQuestionAnswering
import torch

tokenizerQABert = BertTokenizer.from_pretrained("deepset/bert-base-cased-squad2")
modelQABert = BertForQuestionAnswering.from_pretrained("deepset/bert-base-cased-squad2")

Downloading:   0%|          | 0.00/208k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/152 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/508 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/413M [00:00<?, ?B/s]

In [85]:
def answer_with_BERT(question, text):
  inputs = tokenizerQABert(question, text, return_tensors="pt")
  with torch.no_grad():
    outputs = modelQABert(**inputs)
  answer_start_index = outputs.start_logits.argmax()
  answer_end_index = outputs.end_logits.argmax()
  predict_answer_tokens = inputs.input_ids[0, answer_start_index : answer_end_index + 1]
  return tokenizerQABert.decode(predict_answer_tokens)

In [152]:
question, text = "How to check the model?", qa_data.Answers[72]
print(qa_data.Answers[72])

There are a couple of metrics that you can use:
R-squared/Adjusted R-squared: Relative measure of fit. This was explained in a previous answer
F1 Score: Evaluates the null hypothesis that all regression coefficients are equal to zero vs the alternative hypothesis that at least one doesn’t equal zero
RMSE: Absolute measure of fit.


In [172]:
#[CLS] - no answer
print(answer_with_BERT("What is collinearity?", qa_data.Answers[1]))
print(answer_with_BERT("What is support vectors?", qa_data.Answers[7]))

a linear association between two predictors
data points that touch the boundaries of the maximum margin


## Info retrieval func

In [138]:
def find_answer(similiar_question):
  if getApproximateAnswer_cos_sim(similiar_question)[1]<0.5:
    return "Sorry, I didn't get you"
  q = getApproximateAnswer_cos_sim(similiar_question)[2]
  answ = qa_data[qa_data['Questions']==q].iloc[0]['Answers']
  short_answer = answer_with_BERT(similiar_question, answ)
  if short_answer != '[CLS]':
    return short_answer
  else: return answ

In [116]:
print(getApproximateAnswer_cos_sim("What is log regression?"))
print(find_answer('What is log regression?'))

('Logistic regression is also known as the logit model. It is a technique used to forecast the binary outcome from a linear combination of predictor variables.', 0.6745555358646471, 'What is logistic regression?')
['a technique used to forecast the binary outcome from a linear combination of predictor variables']


In [126]:
print(getApproximateAnswer_cos_sim("What is collinearity?"))
print(find_answer('What is collinearity?'))

('Collinearity is a linear association between two predictors. Multicollinearity is a situation where two or more predictors are highly linearly related.\nThis can be problematic because it undermines the statistical significance of an independent variable. While it may not necessarily have a large impact on the model’s accuracy, it affects the variance of the prediction and reduces the quality of the interpretation of the independent variables.\nYou could use the Variance Inflation Factors (VIF) to determine if there is any multicollinearity between independent variables — a standard benchmark is that if the VIF is greater than 5 then multicollinearity exists.', 0.7552678862823228, 'What is collinearity? What is multicollinearity? How do you deal with it?\n')
['a linear association between two predictors']


Compare results with and without summarizer:

In [127]:
print(getApproximateAnswer_cos_sim("Tell me pls, what does linear regression stands for?"), False)
print(find_answer("Tell me pls, what does linear regression stands for?"))

('Regression is a part of supervised ML. Regression models investigate the relationship between a dependent (target) and independent variable (s) (predictor). Here are some common regression models\n\nLinear Regression establishes a linear relationship between target and predictor (s). It predicts a numeric value and has a shape of a straight line.\nPolynomial Regression has a regression equation with the power of independent variable more than 1. It is a curve that fits into the data points.\nRidge Regression helps when predictors are highly correlated (multicollinearity problem). It penalizes the squares of regression coefficients but doesn’t allow the coefficients to reach zeros (uses L2 regularization).\nLasso Regression penalizes the absolute values of regression coefficients and allows some of the coefficients to reach absolute zero (thereby allowing feature selection).', 0.6253195033074085, 'What is regression? Which models can you use to solve a regression problem?') False
['Re

In [128]:
print(getApproximateAnswer_cos_sim("Tell me pls, what does linear regression stands for?"))
print(find_answer("Tell me pls, what does linear regression stands for?"))

('Regression is a part of supervised ML. Regression models investigate the relationship between a dependent (target) and independent variable (s) (predictor). Here are some common regression models\n\nLinear Regression establishes a linear relationship between target and predictor (s). It predicts a numeric value and has a shape of a straight line.\nPolynomial Regression has a regression equation with the power of independent variable more than 1. It is a curve that fits into the data points.\nRidge Regression helps when predictors are highly correlated (multicollinearity problem). It penalizes the squares of regression coefficients but doesn’t allow the coefficients to reach zeros (uses L2 regularization).\nLasso Regression penalizes the absolute values of regression coefficients and allows some of the coefficients to reach absolute zero (thereby allowing feature selection).', 0.6253195033074085, 'What is regression? Which models can you use to solve a regression problem?')
['Regressi

# Part III: Natural Language Generation

In [166]:
# checkpoint 
checkpoint = "microsoft/DialoGPT-medium"
# download and cache tokenizer
tokenizer2 = AutoTokenizer.from_pretrained(checkpoint)
# download and cache pre-trained model
modelNLG = AutoModelForCausalLM.from_pretrained(checkpoint)

# Part IV: Bot Development

In [132]:
import warnings
warnings.filterwarnings("ignore")

In [139]:
# import nltk.data
divide_sent_tok = nltk.data.load('tokenizers/punkt/english.pickle')

In [167]:
class ChatBot():
    def __init__(self):
        self.chat_history_ids = None
        self.bot_input_ids = None
        self.end_chat = False
        self.is_greeting = False
        self.is_question = False
        self.text_input = None
        self.welcome()
        
    def welcome(self):
        print("Initializing ChatBot ...")
        time.sleep(2)
        print('Type any word to start the conversation!\n')
        time.sleep(3)

        
    def user_input(self):
        text = preprocess(str(input("User    >> ")))
        if classify_goodbyes(text.lower()) == 1:
            self.end_chat=True
            print('ChatBot >>  See you soon! Bye!')
            time.sleep(1)
            print('\nQuitting ChatBot ...')
        else:
            if classify_greet(text) == -1:
              self.text_input = text
              if find_answer(self.text_input) != 'Sorry, I didn\'t get you':
                self.is_question = True
            else: 
              self.is_greeting = True

    def bot_greet(self):
        greeting = np.random.choice([
            "Welcome, I am ChatBot, here for your kind service",
            "Hey, Great day! I am your virtual assistant",
            "Hello, it's my pleasure meeting you",
            "Hi, I am a ChatBot. Let's chat!"
        ])
        print("ChatBot >>  " + greeting)
        self.is_greeting = False

    def bot_response(self):
        self.new_user_input_ids = tokenizer2.encode(self.text_input + tokenizer2.eos_token, \
                                                       return_tensors='pt')
        if self.chat_history_ids is not None:
            self.bot_input_ids = torch.cat([self.chat_history_ids, self.new_user_input_ids], dim=-1) 
        else:
            self.bot_input_ids = self.new_user_input_ids
         
        self.chat_history_ids = modelNLG.generate(self.bot_input_ids, max_length=1000, \
                                               pad_token_id=tokenizer2.eos_token_id)
            
        response = tokenizer2.decode(self.chat_history_ids[:, self.bot_input_ids.shape[-1]:][0], \
                               skip_special_tokens=True)
        if response == "":
            response = self.random_response()
        print('ChatBot >>  '+ response)
    
    def bot_answer(self):
      answ = divide_sent_tok.tokenize(find_answer(self.text_input))
      print('ChatBot >>  ', end = ' ')
      for s in answ:
        for i, w in enumerate(s.split()):
          print(w, end = ' ')
          if i%7==6: print()
      self.is_question = False
        
    def random_response(self):
        i = -1
        response = tokenizer2.decode(self.chat_history_ids[:, self.bot_input_ids.shape[i]:][0], \
                               skip_special_tokens=True)
        while response == '':
            i = i-1
            response = tokenizer2.decode(self.chat_history_ids[:, self.bot_input_ids.shape[i]:][0], \
                               skip_special_tokens=True)
        if response.strip() == '?':
            reply = np.random.choice(["I don't know", 
                                     "I am not sure"])
        else:
            reply = np.random.choice(["Great", 
                                      "Fine. What's up?", 
                                      "Okay"
                                     ])
        return reply

In [171]:
bot = ChatBot()
while True:
    bot.user_input()
    if bot.end_chat:
        break
    if bot.is_greeting == False:
      if bot.is_question == True:
        bot.bot_answer()
      else:
        bot.bot_response() 
    else: bot.bot_greet()  

Initializing ChatBot ...
Type any word to start the conversation!

User    >> I've heard a nice word
ChatBot >>  I'm glad you liked it!
User    >> The word inlier. What's that?
ChatBot >>   An inlier is a data observation that 
lies within the rest of the dataset 
and is unusual or an error. Since it lies in the dataset, it 
is typically harder to identify than an 
outlier and requires external data to identify 
them. User    >> hmm
ChatBot >>  I'm glad you liked it!
User    >> Can you predict the future?
ChatBot >>   We want to look at the correlation 
between different observations of y. This measure of correlation is called autocorrelation. 
Autoregressive models are multiple regression models where 
the time-lag series of the original time 
series are treated like multiple independent variables. 
User    >> what is correlation?
ChatBot >>   Covariance is a quantitative measure of the 
extent to which the deviation of one 
variable from its mean matches the deviation 
of the other f